# Externalities

Most environmental problems are caused by externalities. These are costs (or benefits) that are a result of production or consumption that do not go to the producer/consumer, and are therefore
not taken into account by producers or consumers when they make decisions about which quantities to buy or sell. Externalities are production externalities if they are a consequence of production (e.g., carbon emissions if these are not priced or taxed, local air pollution that is a direct consequence of production, water pollution from discharge of waste water, etc.).  Consumption externalities are consequences of consumption (e.g., plastic pollution resulting from consumers disposing of packaging material inappropriately, etc.). Positive externalities occur when production or consumption has positive benefits to someone else in society that are not taken into account by the producer or consumer (e.g., using bees to pollinate crops can have benefits to others around; choosing to vaccinate yourself has benefits to society because you are less likely to infect others, etc). Negative externalities are probably more common (e.g., pollution).

If a production externality occurs, the supply function does not capture the full social cost of production, but only the private cost to the producers. Sticking with perfect competition for simplicity, we can amend the standard supply/demand diagram by adding an additional social marginal cost function which if higher (if there are negative production externalities) or lower (if there are positive production externalities) than the supply function, which is the private marginal cost function. The difference between the private and social marginal cost curves is the size of the externality; the marginal external cost. In most cases, including most pollution scenarios, the marginal external cost increases with production - the higher the level of pollution already is, the worse an additional unit is, but this depends on the externality in question. In the diagram below, we assume a (linearly) increasing marginal external cost of £0.01 x quantity (i.e., if 100 units are being produced, the last one has an external cost of £1.

Since producers (and consumers) do not take the external cost into account, the market, if unregulated, will still reach equilibrium at the intersection between the demand and supply ( = private marginal cost) curve. However, at that point, the quantity produced and consumed is not socially optimal. In the diagram, since there is a negative production externality, the social cost of producting the last unit is higher than the social benefit (in this case, since there are no consumption externalities, the benefit to consumers, e.t. the height of the demand curve). The social optimum is instead at the intersection of the demand curve and the social marginal cost curve, which lies at a smaller quantity and higher price. The externality therefore causes social surplus to be lower than optimal. We call this loss a deadweight loss, as it is a loss that is not compensated for by something else. The size of the deadweight loss is indicated in the figure - it is the difference between the social cost of production and the consumer benefit (the demand curve) between the optimal and market equilibria, i.e., the amount of additional cost to society, over and above the benefit consumers derive from obtaining the product minus the private costs of production. When calculating social surplus, we have to subtract this deadweight loss.

Experiment using the diagram below. When is the deadweight loss large? When is it negligible? Do you understand why this is the case?

In [1]:
# from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import dash_bootstrap_components as dbc

app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dbc.Row([
        # html.H4("Budget constraint"),
        dcc.Graph(id="graph")
    ]),
    
    dbc.Row([
        html.P("Select supply curve intercept:"),
        dcc.Slider(
            id="slider-supply-intercept",
            min=0,
            max=100,
            step=1,
            value=0,
            marks={i: str(i) for i in range(0, 201, 25)}),
    ]),
    dbc.Row([
        html.P("Select supply curve slope:"),
        dcc.Slider(
            id="slider-supply-slope",
            min=0.5,
            max=5,
            step=0.10,
            value=2,
            marks={i: str(i) for i in range(0, 6, 2)})
    ]),
    dbc.Row([
        html.P("Select demand curve intercept:"),
        dcc.Slider(
            id="slider-demand-intercept",
            min=0,
            max=125,
            step=1,
            value=100,
            marks={i: str(i) for i in range(0, 201, 25)},)
    ]),
    dbc.Row([
        html.P("Select demand curve slope:"),
        dcc.Slider(
            id="slider-demand-slope",
            min=-5,
            max=0.1,
            step=0.10,
            value=-3,
            marks={i: str(i) for i in range(-5, 1, 1)})
    ]),
    dbc.Row([
        html.P("Select externality value:"),
        dcc.Slider(
            id="slider-externality-value",
            min=0,
            max=3,
            step=0.1,
            value=2,
            marks={i: str(i) for i in range(0, 4, 1)})
    ])
])

@app.callback(
    Output("graph", "figure"),
    Input("slider-supply-intercept", "value"),
    Input("slider-supply-slope", "value"),
    Input("slider-demand-intercept", "value"),
    Input("slider-demand-slope", "value"),
    Input("slider-externality-value", "value")
)

def plot(supply_intercept, supply_slope, demand_intercept, demand_slope, externality):
    def supply_function(supply_slope, supply_intercept):
        x_ = np.arange(0.1, 100., 0.1)
        y_ = []
        for x in x_:
            y = supply_slope * x + supply_intercept
            y_.append(y)
        df_plot = pd.DataFrame({'Quantity': x_, 'Price': y_})
        return df_plot
    def demand_function(demand_slope, demand_intercept):
        x_ = np.arange(0.1, 100., 0.1)
        y_ = []
        for x in x_:
            y = demand_slope * x + demand_intercept
            y_.append(y)
        df_plot = pd.DataFrame({'Quantity': x_, 'Price': y_})
        return df_plot
    def social_cost_function(supply_slope, supply_intercept, externality):
        x_ = np.arange(0.1, 100., 0.1)
        y_ = []
        for x in x_:
            y = supply_slope * x * externality + supply_intercept
            y_.append(y)
        df_plot = pd.DataFrame({'Quantity': x_, 'Price': y_})
        return df_plot
    def private_quantity_calc(supply_slope, supply_intercept, demand_slope, demand_intercept):
        quantity = (demand_intercept - supply_intercept) / (supply_slope - demand_slope)
        return quantity
    def private_price_calc(quantity, supply_slope, supply_intercept):
        price = supply_slope * quantity + supply_intercept
        return price
    def social_quantity_calc(supply_slope, supply_intercept, demand_slope, demand_intercept, externality):
        quantity = (demand_intercept - supply_intercept) / (supply_slope * externality - demand_slope)
        return quantity
    def social_price_calc(quantity, supply_slope, supply_intercept, externality):
        price = supply_slope * quantity * externality + supply_intercept
        return price
    def deadweight_calc(private_quantity, social_quantity, private_price, social_price_at_private_quantity):
        deadweight = (private_quantity - social_quantity) * (social_price_at_private_quantity - private_price) / 2
        return deadweight

    df_supply = supply_function(supply_slope, supply_intercept)
    fig1 = px.line(df_supply, x='Quantity', y='Price', range_x=[0,50], range_y=[0,100])
    fig1.update_traces(line_color='blue', name='Supply curve', showlegend=True)
    df_demand = demand_function(demand_slope, demand_intercept)
    fig2 = px.line(df_demand, x='Quantity', y='Price', range_x=[0,50], range_y=[0,100])
    fig2.update_traces(line_color='green', name='Demand curve', showlegend=True)
    df_social = social_cost_function(supply_slope, supply_intercept, externality)
    fig3 = px.line(df_social, x='Quantity', y='Price', range_x=[0,50], range_y=[0,100])
    fig3.update_traces(line_color='black', line_dash="dash", name='Social cost curve', showlegend=True)
    fig = go.Figure(data=fig1.data + fig2.data + fig3.data, layout = fig1.layout)

    private_quantity = private_quantity_calc(supply_slope, supply_intercept, demand_slope, demand_intercept)
    private_price = private_price_calc(private_quantity, supply_slope, supply_intercept)
    social_quantity = social_quantity_calc(supply_slope, supply_intercept, demand_slope, demand_intercept, externality)
    social_price = social_price_calc(social_quantity, supply_slope, supply_intercept, externality)
    social_price_at_private_quantity = social_price_calc(private_quantity, supply_slope, supply_intercept, externality)
    deadweight_value = deadweight_calc(private_quantity, social_quantity, private_price, social_price_at_private_quantity)

    fig_shape1 = go.Figure(go.Scatter(x=[private_quantity, private_quantity, social_quantity], y=[private_price, social_price_at_private_quantity, social_price], line=dict(color="LightGreen"), fill="toself"))
    deadweight_text = "Deadweight: " + str(round(deadweight_value, 2))
    fig_shape1.update_traces(name=deadweight_text)

    fig = go.Figure(data=fig.data + fig_shape1.data, layout = fig1.layout)

    # fig.add_shape(type="line", x0=0, y0=private_price, x1=private_quantity, y1=private_price, line=dict(color="red", width=3, dash="dash"))
    # fig.add_shape(type="line", x0=private_quantity, y0=0, x1=private_quantity, y1=private_price, line=dict(color="LightGrey", width=3, dash="dash"))

    # fig.add_shape(type="line", x0=0, y0=social_price, x1=social_quantity, y1=social_price, line=dict(color="red", width=3, dash="dash"))
    # fig.add_shape(type="line", x0=social_quantity, y0=0, x1=social_quantity, y1=social_price, line=dict(color="LightGrey", width=3, dash="dash"))

    # fig.add_shape(type="line", x0=0, y0=social_price_at_private_quantity, x1=private_quantity, y1=social_price_at_private_quantity, line=dict(color="red", width=3, dash="dash"))

    return fig

app.run_server(mode='inline', port=8061, jupyter_height=950)

## Correcting an externality with taxes

Since externalities imply that the market equilibrium does not lead to the maximum social welfare, there is a potential for policy to increase welfare. There are a number of ways to do this, and how they work depend on whether it is possible to address the cause of the externality (e.g., whether it is possible to change production processes to bring the marginal external cost curve itself down) or whether it is only possible to reduce quantities produced. We will analyse the second case with a different set of models, but in this notebook, we focus on a case where the marginal external cost curve cannot be changed - i.e., production is unavoidably linked to pollution. In this case, we will still like to make sure that social welfare is maximised. We can do this with taxation that internalises the externality, i.e. makes the producers pay for their pollution and therefore bringing the external costs into their optimisation problem. 

So, ideally, we’d like to increase the private marginal cost curve such that it matches the social marginal cost curve. Hower, in practice, it is usually difficult to set tax rates that vary with the level of production (and therefore also vary across producers, if there are multiple). Constant per-unit taxes are much easier - these would force producers to pay a fixed amount for each unit they produce, and therefore shift the supply curve straight up, without changing its slope. The good news is that this can still achieve the social optimum if we set the tax rate correctly. Using the diagram below, experiment with the tax rate (if you choose a negative value for the tax, this is effectively setting a subsidy). When is the social optimum achieved? (i.e. when is the deadweight loss eliminated?). Why?

In [2]:
# from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import dash_bootstrap_components as dbc

app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    dbc.Row([
        # html.H4("Budget constraint"),
        dcc.Graph(id="graph")
    ]),
    
    dbc.Row([
        html.P("Select supply curve intercept:"),
        dcc.Slider(
            id="slider-supply-intercept",
            min=0,
            max=100,
            step=1,
            value=0,
            marks={i: str(i) for i in range(0, 201, 25)}),
    ]),
    dbc.Row([
        html.P("Select supply curve slope:"),
        dcc.Slider(
            id="slider-supply-slope",
            min=0.5,
            max=5,
            step=0.10,
            value=2,
            marks={i: str(i) for i in range(0, 6, 2)})
    ]),
    dbc.Row([
        html.P("Select demand curve intercept:"),
        dcc.Slider(
            id="slider-demand-intercept",
            min=0,
            max=125,
            step=1,
            value=100,
            marks={i: str(i) for i in range(0, 201, 25)},)
    ]),
    dbc.Row([
        html.P("Select demand curve slope:"),
        dcc.Slider(
            id="slider-demand-slope",
            min=-5,
            max=0.1,
            step=0.10,
            value=-3,
            marks={i: str(i) for i in range(-5, 1, 1)})
    ]),
    dbc.Row([
        html.P("Select externality value:"),
        dcc.Slider(
            id="slider-externality-value",
            min=0,
            max=3,
            step=0.1,
            value=2,
            marks={i: str(i) for i in range(0, 4, 1)})
    ]),
    dbc.Row([
        html.P("Select tax per unit of quantity:"),
        dcc.Slider(
            id="slider-tax-value",
            min=0,
            max=50,
            step=0.1,
            value=0,
            marks={i: str(i) for i in range(0, 51, 5)})
    ]),
])

@app.callback(
    Output("graph", "figure"),
    Input("slider-supply-intercept", "value"),
    Input("slider-supply-slope", "value"),
    Input("slider-demand-intercept", "value"),
    Input("slider-demand-slope", "value"),
    Input("slider-externality-value", "value"),
    Input("slider-tax-value", "value")
)

def plot(supply_intercept, supply_slope, demand_intercept, demand_slope, externality, tax):
    def supply_function(supply_slope, supply_intercept, tax):
        x_ = np.arange(0.1, 100., 0.1)
        y_ = []
        for x in x_:
            y = supply_slope * x + supply_intercept + tax
            y_.append(y)
        df_plot = pd.DataFrame({'Quantity': x_, 'Price': y_})
        return df_plot
    def demand_function(demand_slope, demand_intercept):
        x_ = np.arange(0.1, 100., 0.1)
        y_ = []
        for x in x_:
            y = demand_slope * x + demand_intercept
            y_.append(y)
        df_plot = pd.DataFrame({'Quantity': x_, 'Price': y_})
        return df_plot
    def social_cost_function(supply_slope, supply_intercept, externality):
        x_ = np.arange(0.1, 100., 0.1)
        y_ = []
        for x in x_:
            y = supply_slope * x * externality + supply_intercept
            y_.append(y)
        df_plot = pd.DataFrame({'Quantity': x_, 'Price': y_})
        return df_plot
    def private_cost_function(supply_slope, supply_intercept):
        x_ = np.arange(0.1, 100., 0.1)
        y_ = []
        for x in x_:
            y = supply_slope * x + supply_intercept
            y_.append(y)
        df_plot = pd.DataFrame({'Quantity': x_, 'Price': y_})
        return df_plot
    def private_quantity_calc(supply_slope, supply_intercept, demand_slope, demand_intercept, tax):
        quantity = (demand_intercept - supply_intercept - tax) / (supply_slope - demand_slope)
        return quantity
    def private_price_calc(quantity, supply_slope, supply_intercept, tax):
        price = supply_slope * quantity + supply_intercept + tax
        return price
    def social_quantity_calc(supply_slope, supply_intercept, demand_slope, demand_intercept, externality):
        quantity = (demand_intercept - supply_intercept) / (supply_slope * externality - demand_slope)
        return quantity
    def social_price_calc(quantity, supply_slope, supply_intercept, externality):
        price = supply_slope * quantity * externality + supply_intercept
        return price
    def deadweight_calc(private_quantity, social_quantity, private_price, social_price_at_private_quantity):
        deadweight = (private_quantity - social_quantity) * (social_price_at_private_quantity - private_price) / 2
        return deadweight

    df_supply = supply_function(supply_slope, supply_intercept, tax)
    fig1 = px.line(df_supply, x='Quantity', y='Price', range_x=[0,50], range_y=[0,100])
    fig1.update_traces(line_color='blue', name='Supply curve', showlegend=True)
    df_demand = demand_function(demand_slope, demand_intercept)
    fig2 = px.line(df_demand, x='Quantity', y='Price', range_x=[0,50], range_y=[0,100])
    fig2.update_traces(line_color='green', name='Demand curve', showlegend=True)
    df_social = social_cost_function(supply_slope, supply_intercept, externality)
    fig3 = px.line(df_social, x='Quantity', y='Price', range_x=[0,50], range_y=[0,100])
    fig3.update_traces(line_color='black', line_dash="dash", name='Social cost curve', showlegend=True)
    df_private = private_cost_function(supply_slope, supply_intercept)
    fig4 = px.line(df_private, x='Quantity', y='Price', range_x=[0,50], range_y=[0,100])
    fig4.update_traces(line_color='LightBlue', line_dash="dash", name='Private cost curve', showlegend=True)
    fig = go.Figure(data=fig1.data + fig2.data + fig3.data + fig4.data, layout = fig1.layout)

    private_quantity = private_quantity_calc(supply_slope, supply_intercept, demand_slope, demand_intercept, tax)
    private_price = private_price_calc(private_quantity, supply_slope, supply_intercept, tax)
    social_quantity = social_quantity_calc(supply_slope, supply_intercept, demand_slope, demand_intercept, externality)
    social_price = social_price_calc(social_quantity, supply_slope, supply_intercept, externality)
    social_price_at_private_quantity = social_price_calc(private_quantity, supply_slope, supply_intercept, externality)
    deadweight_value = deadweight_calc(private_quantity, social_quantity, private_price, social_price_at_private_quantity)

    fig_shape1 = go.Figure(go.Scatter(x=[private_quantity, private_quantity, social_quantity], y=[private_price, social_price_at_private_quantity, social_price], line=dict(color="LightGreen"), fill="toself"))
    deadweight_text = "Deadweight: " + str(round(deadweight_value, 2))
    fig_shape1.update_traces(name=deadweight_text)

    fig = go.Figure(data=fig.data + fig_shape1.data, layout = fig1.layout)

    # fig.add_shape(type="line", x0=0, y0=private_price, x1=private_quantity, y1=private_price, line=dict(color="red", width=3, dash="dash"))
    # fig.add_shape(type="line", x0=private_quantity, y0=0, x1=private_quantity, y1=private_price, line=dict(color="LightGrey", width=3, dash="dash"))

    # fig.add_shape(type="line", x0=0, y0=social_price, x1=social_quantity, y1=social_price, line=dict(color="red", width=3, dash="dash"))
    # fig.add_shape(type="line", x0=social_quantity, y0=0, x1=social_quantity, y1=social_price, line=dict(color="LightGrey", width=3, dash="dash"))

    # fig.add_shape(type="line", x0=0, y0=social_price_at_private_quantity, x1=private_quantity, y1=social_price_at_private_quantity, line=dict(color="red", width=3, dash="dash"))

    return fig

app.run_server(mode='inline', port=8061, jupyter_height=950)